In [ ]:
import random
import numpy as np

## GridWorld 클래스

In [ ]:
class GridWorld():
    def __init__(self):
        self.x = 0
        self.y = 0

    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션 : 아래쪽
        if a==0:
            self.move_left()
        elif a==1:
            self.move_up()
        elif a==2:
            self.move_right()
        elif a==3:
            self.move_down()
    
        reward = -1 #보상은 1로 고정
        done = self.is_done()
        return (self.x, self.y), reward, done

    def move_left(self):
        if self.y==0:
            pass
        elif self.y==3 and self.x in [0, 1, 2]:
            pass
        elif self.y==5 and self.x in [2, 3, 4]:
            pass
        else:
            self.y -=1

    def move_right(self):
        if self.y==1 and self.x in [0, 1, 2]:
            pass
        elif self.y==3 and self.x in [2, 3, 4]:
            pass
        elif self.y==6:
            pass
        else:
            self.y +=1

    def move_up(self):
        if self.x==0:
            pass
        elif self.x==3 and self.y==2:
            pass
        else:
            self.x -= 1

    def move_down(self):
        if self.x==4:
            pass
        elif self.x==1 and self.y==4:
            pass
        else:
            self.x+=1
    
    def is_done(self):
        if self.x==4 and self.y==6: # 목표지점인 (4, 6)에 도달하면 끝난다.
            return True
        else:
            return False
    
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

## QAgent 클래스

In [ ]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) #q밸류를 저장하는 변수, 모두 0이고 3차원임
        self.eps = 0.9 #엡실론(감쇠는 미적용?)
        self.alpha = 0.01 
    
    def select_action(self, s):
        # 엡실론-그리디로 액션 선택
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
        return action
    
    def update_table(self, history):
        # 한 에피소드에 해당하는 history를 입력으로 받아 q 테이블의 값을 업데이트 한다.
        cum_reward = 0
        for transition in history[::-1]:
            # print(transition) # 디버깅용 프린트
            s, a, r , s_prime = transition
            x, y = s  #몬테카를로 방식 업데이트
            self.q_table[x,y,a] = self.q_table[x,y,a] + self.alpha * (cum_reward - self.q_table[x,y,a])
            cum_reward = cum_reward + r

    def anneal_eps(self):
        self.eps -= 0.03 #감쇠텀 여기있네 한번 호출될때마다 0.03씩 내려감
        self.eps = max(self.eps, 0.1) #최소 0.1은 보장
    
    def show_table(self):
        # 학습이 각 위치에서 어느 액션의 q값이 가장 높았는지 보여주는 함수 (뭔소리?)
        q_lst = self.q_table.tolist()  #넘파이어레이 리스트로 바꿔줌
        data = np.zeros((5, 7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

## 메인 함수

In [ ]:
def main():
    env = GridWorld()
    agent = QAgent()
    episode = 1000

    for n_epi in range(episode): #1000번 에피소드
        done = False
        history = []

        s = env.reset()
        while not done: # 한 에피소드가 끝날때 까지
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            history.append((s, a, r, s_prime))
            s = s_prime
        agent.update_table(history) #히스토리를 이용하여 에이전트를 업데이트
        agent.anneal_eps()

    agent.show_table()
    

In [ ]:
main()

[[2. 3. 0. 2. 2. 3. 0.]
 [3. 3. 0. 2. 1. 3. 3.]
 [3. 3. 0. 1. 0. 3. 3.]
 [3. 2. 3. 1. 0. 2. 3.]
 [2. 1. 2. 1. 0. 2. 0.]]


## SARSA (MC대신 TD 사용)

In [ ]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) #q밸류를 저장하는 변수, 모두 0이고 3차원임
        self.eps = 0.9 
    
    def select_action(self, s):
        # 엡실론-그리디로 액션 선택
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
        return action
    
    # 나머지는 같고 table을 업데이트 하는 방식만 바꿔준당
    def update_table(self, transition):
        s, a, r , s_prime = transition  #history 대신 바로 transition을 사용
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime) # s'에서 선택할 액션
        #SARSA 업데이트 식을 이용
        self.q_table[x,y,a] = self.q_table[x,y,a] + 0.1*(r + self.q_table[next_x, next_y, a_prime] - self.q_table[x,y,a])

    def anneal_eps(self):
        self.eps -= 0.03 #감쇠텀 여기있네 한번 호출될때마다 0.03씩 내려감
        self.eps = max(self.eps, 0.1) #최소 0.1은 보장
    
    def show_table(self):
        # 학습이 각 위치에서 어느 액션의 q값이 가장 높았는지 보여주는 함수 (뭔소리?)
        q_lst = self.q_table.tolist()  #넘파이어레이 리스트로 바꿔줌
        data = np.zeros((5, 7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [ ]:
def main():
    env = GridWorld()
    agent = QAgent()
    episode = 1000

    for n_epi in range(episode): #1000번 에피소드
        done = False

        s = env.reset()
        while not done: # 한 에피소드가 끝날때 까지
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            agent.update_table((s, a, r, s_prime))
            s = s_prime
        agent.anneal_eps()

    agent.show_table()
'''
history에 리스트를 모아서 한번에 바꿔준 이전과 다르게
매번 update_table을 호출해서 바꿔주는 것으로 TD, MC 차이를 확인가능
'''

'\nhistory에 리스트를 모아서 한번에 바꿔준 이전과 다르게\n매번 update_table을 호출해서 바꿔주는 것으로 TD, MC 차이를 확인가능\n'

In [ ]:
main()

[[2. 3. 0. 2. 2. 2. 3.]
 [3. 3. 0. 2. 2. 3. 3.]
 [3. 3. 0. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 3. 3.]
 [3. 2. 2. 1. 0. 2. 0.]]


## Q러닝 QAgent

Class GridWorld () <- 환경이므로 건드릴 필요 없슴


In [ ]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) #q밸류를 저장하는 변수, 모두 0이고 3차원임
        self.eps = 0.9 
    
    def select_action(self, s):
        # 엡실론-그리디로 액션 선택
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
        return action

    def update_table(self, transition):
        s, a, r , s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime) 
        #SARSA대신 Q러닝 업데이트 식을 이용
        self.q_table[x,y,a] = self.q_table[x,y,a] + 0.1*(r + np.argmax(self.q_table[next_x, next_y, :]) - self.q_table[x,y,a])

    def anneal_eps(self):
        self.eps -= 0.01 #q러닝에서는 epsilon이 더 천천히 줄어들도록 함(왜?)
        self.eps = max(self.eps, 0.2) # 0.2도 더 크다(왜?)

    def show_table(self):
        q_lst = self.q_table.tolist()  
        data = np.zeros((5, 7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [ ]:
main()

[[2. 2. 0. 2. 3. 0. 2.]
 [1. 1. 0. 2. 3. 0. 1.]
 [2. 2. 0. 1. 0. 3. 0.]
 [1. 2. 3. 0. 0. 1. 0.]
 [2. 2. 3. 0. 0. 1. 0.]]
